> Algumas questões foram alteradas depois do envio na Udemy

##### Parte 1 > Realizando import's e iniciando pyspark

In [54]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StringType
import random
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from functools import reduce

In [55]:
# Nesta etapa, adicoine código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. 
# Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

spark = SparkSession.builder.master("local[*]").appName("Tarefa").getOrCreate()
df_nomes = spark.read.csv('../Tarefa3/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



##### Parte 2 > Renomear coluna, imprimir o schema e mostrar 10 linhas

In [56]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(10)


root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



##### Parte 3 > Adicionar nova coluna chamada Escolaridade

In [57]:
escolaridade = ['Fundamental', 'Médio', 'Superior']

df_nomes = df_nomes.withColumn('Escolaridade', udf(f=lambda: escolaridade[random.randint(0,2)],returnType=StringType())())

##### Parte 4 > Adicionar países da América do Sul


In [58]:
paises = ["Brasil", "Argentina", "Colômbia", "Peru", "Venezuela", "Chile", "Equador",
            "Bolívia", "Paraguai", "Uruguai", "Guiana", "Suriname", "Guiana Francesa"]

df_nomes = df_nomes.withColumn('Pais', udf(f=lambda: paises[random.randint(0,12)],returnType=StringType())())

##### Parte 5 > Adicione nova coluna chamada AnoNascimento

In [59]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("CAST(FLOOR(RAND() * (66) + 1945) AS INT)"))

##### Parte 6 > Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século

In [60]:
df_select = df_nomes.select('*').where(df_nomes.AnoNascimento >= 2000)
df_select.show(10)

+---------------+------------+---------+-------------+
|          Nomes|Escolaridade|     Pais|AnoNascimento|
+---------------+------------+---------+-------------+
|    Frank Wiley|    Superior| Suriname|         2002|
|     Ned Tester| Fundamental|    Chile|         2008|
|  George Miller| Fundamental|Argentina|         2009|
|  Evelyn Shaver| Fundamental| Paraguai|         2008|
|   Ida Randazzo|    Superior|Argentina|         2007|
|   Curtis Sutch|    Superior|  Bolívia|         2002|
|Pamela Banuelos| Fundamental|     Peru|         2003|
|   Douglas Boyd| Fundamental|  Equador|         2010|
| Roger Guenther|       Médio|Argentina|         2003|
|  Fannie Taylor|       Médio| Paraguai|         2009|
+---------------+------------+---------+-------------+
only showing top 10 rows



##### Parte 7 > Usando Spark SQL repita o processo da Pergunta 6.

In [61]:
df_nomes.createOrReplaceTempView('pessoas')
spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000').show(10)


+---------------+------------+---------+-------------+
|          Nomes|Escolaridade|     Pais|AnoNascimento|
+---------------+------------+---------+-------------+
|    Frank Wiley|    Superior|Venezuela|         2002|
|     Ned Tester|       Médio|  Equador|         2008|
|  George Miller|       Médio|   Brasil|         2009|
|  Evelyn Shaver| Fundamental|Venezuela|         2008|
|   Ida Randazzo|    Superior|    Chile|         2007|
|   Curtis Sutch| Fundamental|  Equador|         2002|
|Pamela Banuelos|    Superior| Colômbia|         2003|
|   Douglas Boyd|    Superior| Suriname|         2010|
| Roger Guenther|       Médio| Paraguai|         2003|
|  Fannie Taylor|    Superior| Suriname|         2009|
+---------------+------------+---------+-------------+
only showing top 10 rows



##### Parte 8 > Conte o número de pessoas que são da geração Millennials


In [62]:
m = df_nomes.select('*').where(df_nomes.AnoNascimento.between(1980, 1994)).count()
print(m)


2271560


##### Parte 9 > Repita o processo da Pergunta 8 utilizando Spark SQL

In [63]:
spark.sql('SELECT COUNT(*) as Contagem FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994').show()

+--------+
|Contagem|
+--------+
| 2271560|
+--------+



##### Parte 10 > Obtenha a quantidade de pessoas de cada país para uma das gerações

In [64]:
sqlContext = SQLContext(spark)

babys = sqlContext.sql("SELECT Pais, 'Baby Boomers' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais")

ger_x = sqlContext.sql("SELECT Pais, 'Geração X' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1965 AND 1979\
                        GROUP BY Pais")

millenials = sqlContext.sql("SELECT Pais, 'Millennials' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1980 AND 1994\
                        GROUP BY Pais")

ger_z = sqlContext.sql("SELECT Pais, 'Geração Z' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1995 AND 2015\
                        GROUP BY Pais")

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [65]:
geracoes = [babys, ger_x, millenials, ger_z]
geracao_union = reduce(DataFrame.unionAll, geracoes)

In [70]:
geracao_union.orderBy(asc("Pais"),"Geracao", asc("Quantidade")).show(geracao_union.count(), False)

+---------------+------------+----------+
|Pais           |geracao     |Quantidade|
+---------------+------------+----------+
|Argentina      |Baby Boomers|233121    |
|Argentina      |Geração X   |175221    |
|Argentina      |Geração Z   |186355    |
|Argentina      |Millennials |175157    |
|Bolívia        |Baby Boomers|233269    |
|Bolívia        |Geração X   |175421    |
|Bolívia        |Geração Z   |186300    |
|Bolívia        |Millennials |173779    |
|Brasil         |Baby Boomers|233564    |
|Brasil         |Geração X   |175287    |
|Brasil         |Geração Z   |186305    |
|Brasil         |Millennials |173993    |
|Chile          |Baby Boomers|233356    |
|Chile          |Geração X   |174558    |
|Chile          |Geração Z   |186484    |
|Chile          |Millennials |175157    |
|Colômbia       |Baby Boomers|234078    |
|Colômbia       |Geração X   |175174    |
|Colômbia       |Geração Z   |186555    |
|Colômbia       |Millennials |175420    |
|Equador        |Baby Boomers|2326